In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()

SUBSCRIPTION = "<your-subscription-id>"
RESOURCE_GROUP = "<your-resource-group-name>"
WS_NAME = "<your-workspace-name>"

ml_client = MLClient(
    credential=credential,
    subscription_id=SUBSCRIPTION,
    resource_group_name=RESOURCE_GROUP,
    workspace_name=WS_NAME,
)

In [ ]:
from azure.ai.ml import command
from azure.ai.ml import Input
data_asset = ml_client.data.get("spam-email-data", version="cleaned3")

registered_model_name = "email_spam_model"

job = command(
    inputs=dict(
        data=Input(
            type="uri_file",
            path=data_asset.path,
        ),
        test_train_ratio=0.2,
        registered_model_name=registered_model_name,
    ),
    code="./src/",  # location of source code
    command="python main.py --data ${{inputs.data}} --test_train_ratio ${{inputs.test_train_ratio}} --registered_model_name ${{inputs.registered_model_name}}",
    environment="spam-scikit-learn@latest",
    display_name="email_spam_prediction",
)

In [ ]:
ml_client.create_or_update(job)